# Merge signals with CRSP
Example of merging signals (a.k.a. firm-level characteristic) with CRSP monthly returns and construct portfolio sorts. The key is to lag the signals by one month. You'll need to have a WRDS account to download the CRSP returns.

In [1]:
# == Setup ==

# load packages
import pandas as pd
import openassetpricing as oap
import wrds

# connect to WRDS
wrds_conn = wrds.Connection()

# initialize OpenAP
openap = oap.OpenAP()

# desired signal list
signal_list = ['BM', 'AssetGrowth']

WRDS recommends setting up a .pgpass file.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


# Download data

Let's download select signals and a sample of CRSP returns. Note that in the `dl_signal` call, we do not ask for the signals to be "signed." We'll return to this in the Sanity Check section.

In [2]:
# download select signals
signals = openap.dl_signal('pandas', signal_list)


Data is downloaded: 13s


In [3]:
# download crsp returns
# use 1980 to 1989 for quick dl and for similarity to original paper periods
crsp = wrds_conn.raw_sql("""select permno, date, ret*100 as ret
                        from crsp.msf 
                        where date>='01/01/1980' and date<='12/31/1989'""", 
                     date_cols=['date'])

<span style="color:orange;font-weight:bold">Optional:</span> you may want to screen for common stocks and standard exchanges in the above. This filter matters little for this older sample, but in more recent years this filter is helpful for removing ETFs.

# Lag signals and merge

To lag signals, you can just add one month to the `yyyymm` column for the signals. For simplicity, let's fill in the day of the new variable `date` as the 28th (the signals are assumed to be available for trading at the end of the month). You can keep around `yyyymm` as `yyyymm_signals` for sanity checks. 

In [4]:
# lag signals
signals_lag = signals.copy()
signals_lag = signals_lag.rename(columns={'yyyymm': 'yyyymm_signals'})
signals_lag['date'] = (
    pd.to_datetime(signals_lag['yyyymm_signals'].astype(str) + '28', format='%Y%m%d') +
    pd.DateOffset(months=1))

signals_lag.head()

,permno,yyyymm_signals,BM,AssetGrowth,date
0,10000,198704,-2.895161,NaN,1987-05-28
1,10000,198705,-2.895161,NaN,1987-06-28
2,10001,198612,-0.104958,NaN,1987-01-28
3,10001,198701,-0.104958,NaN,1987-02-28
4,10001,198702,-0.104958,NaN,1987-03-28


To merge with monthly CRSP returns, convert the date to the 28th, and left join returns onto the lagged signals. This method preserves as much of the data as possible and makes missing data issues transparent. You should see that `date` is one month after `yyyymm_signals`.

In [5]:
# convert crsp dates to the 28th of the month
crsp['date'] = pd.to_datetime(crsp['date'].dt.strftime('%Y%m') + '28', format='%Y%m%d')

# left join returns onto signals
crsp_signals = pd.merge(crsp, signals_lag, on=['permno', 'date'], how='left')

crsp_signals.head()

,permno,date,ret,yyyymm_signals,BM,AssetGrowth
0,10006,1980-01-28,21.1679,197912.0,0.105462,0.173800
1,10057,1980-01-28,28.2297,197912.0,-0.043609,0.078283
2,10058,1980-01-28,0.0000,NaN,NaN,NaN
3,10065,1980-01-28,9.9216,NaN,NaN,NaN
4,10103,1980-01-28,8.3333,197912.0,1.027274,NaN


# Sanity checks
Let's check some magnitudes, make sure they're reasonable. Portfolio sorts are usually the best way to check this. First B/M:

In [6]:
n = 5  # define the number of portfolios
signalname = 'BM' # select signal

# process signal data, assign to portfolios
temp = crsp_signals[['permno', 'date', 'ret', signalname]].copy()
temp['port'] = temp.groupby('date')[signalname].transform(
    lambda x: pd.qcut(x, n, labels=list(range(1, n+1)))
) 

# calculate number of stocks and mean return by date and portfolio
port = temp.groupby(['date', 'port'], observed=False).agg(
    nstock=('permno', 'count'),
    ret=('ret', 'mean')
).reset_index()

# calculate mean return, sd return, and mean number of stocks by portfolio
port_summary = port.groupby('port', observed=False).agg({
    'ret': ['mean', 'std'],
    'nstock': 'mean',
    'date': ['min', 'max']
})

port_summary

ret                nstock       date           
          mean       std        mean        min        max
port                                                      
1     0.392821  7.011399  840.391667 1980-01-28 1989-12-28
2     1.190261  6.036643  839.850000 1980-01-28 1989-12-28
3     1.480457  5.343920  839.775000 1980-01-28 1989-12-28
4     1.546331  4.788628  839.850000 1980-01-28 1989-12-28
5     1.822715  4.929924  840.233333 1980-01-28 1989-12-28

So B/M long-short quintiles earned about 140 bps per month. This absurdly large, from a naive perspective. But these returns use tiny stocks (see the nstock count),  occur before the internet (see the date min and max), and are quite concentrated in the short leg (see return gap between ports 1 and 2). Overall, this magnitude matches what is found in the anomalies lit (McLean and Pontiff 2016; Chen and Zimmermann 2021, CFR).

Now let's check on AssetGrowth

In [7]:
n = 5  # define the number of portfolios
signalname = 'AssetGrowth' # select signal

# process signal data, assign to portfolios
temp = crsp_signals[['permno', 'date', 'ret', signalname]].copy()
temp['port'] = temp.groupby('date')[signalname].transform(
    lambda x: pd.qcut(x, n, labels=list(range(1, n+1)))
) 

# calculate number of stocks and mean return by date and portfolio
port = temp.groupby(['date', 'port'], observed=False).agg(
    nstock=('permno', 'count'),
    ret=('ret', 'mean')
).reset_index()

# calculate mean return, sd return, and mean number of stocks by portfolio
port_summary = port.groupby('port', observed=False).agg({
    'ret': ['mean', 'std'],
    'nstock': 'mean',
    'date': ['min', 'max']
})

port_summary

ret                nstock       date           
          mean       std        mean        min        max
port                                                      
1     1.411934  6.068294  816.050000 1980-01-28 1989-12-28
2     1.616452  4.959080  815.483333 1980-01-28 1989-12-28
3     1.560212  4.714884  815.466667 1980-01-28 1989-12-28
4     1.405766  5.254997  815.483333 1980-01-28 1989-12-28
5     0.671774  6.426844  815.875000 1980-01-28 1989-12-28

The return spread of 74 bps per month for long-short quintiles is once again similar to the anomalies. As usual, the short leg is super critical. Nearly all of the returns are found by shorting portfolio 5---at least during the 1980s and in these tiny stocks.

But unlike B/M, high asset growth means *low* returns going forward. We can check that this makes sense by checking the Signal Documentation.

In [8]:
# download signal doc
signaldoc = openap.dl_signal_doc('pandas')

# increase display width
pd.set_option('display.max_colwidth', 150)

# show detailed definition for Acronyms in signal list
signaldoc[signaldoc['Acronym'].isin(signal_list)][['Acronym', 'Authors', 'Year', 'Sign', 'Detailed Definition']]


,Acronym,Authors,Year,Sign,Detailed Definition
7,AssetGrowth,"Cooper, Gulen and Schill",2008,-1,Annual growth rate of total assets (at)
11,BM,Stattman,1980,1,Log of tangible book equity (ceqt) over market equity matched at FYE


According to the signal doc, Cooper, Gulen, and Schill (2008) find a negative relationship between AssetGrowth and future returns, in contrast to the positive sign for BM from Stattman 1980. If you know this literature well, you can see the signs by the Detailed Definitions.

One can alternatively download signals such that high signals mean high future returns, by using the `signed` argument in `dl_signal`:

In [9]:
# download signed signals
signals_signed = openap.dl_signal('pandas', signal_list, signed=True)


Data is downloaded: 13s


In [18]:
# compare signed and unsigned signals
pd.merge(
    signals[signals['permno']==10001].drop_duplicates(subset=signal_list),
    signals_signed,
    on=['permno', 'yyyymm']
).head()

,permno,yyyymm,BM_x,AssetGrowth_x,BM_y,AssetGrowth_y
0,10001,198612,-0.104958,NaN,-0.104958,NaN
1,10001,198712,-0.081024,-0.038474,-0.081024,0.038474
2,10001,198812,-0.117622,-0.003058,-0.117622,0.003058
3,10001,198912,-0.103032,0.582020,-0.103032,-0.582020
4,10001,199012,-0.336636,0.017021,-0.336636,-0.017021


So using `signed=True`, the sign of BM is unchanged, but the sign of AssetGrowth is flipped.